# imports

In [ ]:
import os
import json
import dateutil
import datetime
import csv
import numpy as np
import SensorThings as st

# settings

In [ ]:
reloaddata = True

datastream_id = "testdatastream:PM10"
starttime = "2018-11-15T00:00:00.000Z"
endtime = "2018-11-16T23:59:59.999Z"

st_url = "http://smartaqnet.teco.edu:8080/FROST-Server"

# get and save data

In [ ]:
where = "/v1.0/Datastreams('" + datastream_id + "')/Observations"

qdata = [
    "$expand=" +
        "Datastream($select=id)," + 
        "FeatureOfInterest($select=id,feature)",
    
    "$filter=" +
        "phenomenonTime ge " + starttime + " and "
        "phenomenonTime le " + endtime,
    
    "$select=id,result,phenomenonTime"
]

filename = "entities.json"

if (not os.path.isfile(filename)) or reloaddata:
    entities = st.GetEntities(st_url, where, qdata)
    with open(filename, 'w') as outfile:
        json.dump(entities, outfile)
else:
    with open(filename, 'r') as infile:
        entities = json.load(infile)
        
data = [
    [
        dateutil.parser.parse(elem['phenomenonTime']).timestamp(),
        elem['phenomenonTime'],
        elem["@iot.id"],
        elem["FeatureOfInterest"]['@iot.id']
    ]
    for elem in entities
]

data.sort(key=lambda x: x[0])
data_cols = list(zip(*data))

# load positions

In [ ]:
with open("position.csv", "r") as file:
    reader = csv.reader(file, delimiter=";")
    position_header = next(reader, None)
    
    # -----------------------------------------
    # TODO: modify your GPS data reader here!!!
    # -----------------------------------------
    position = [
        [
            dateutil.parser.parse(elem[12]).timestamp(),
            elem[12],
            # longitude
            float(elem[9]),
            # latitude
            float(elem[10]),
            # elevation
            float(elem[11])
        ]
        for elem in reader
    ]

position.sort(key=lambda x: x[0])
position_cols = list(zip(*position))

# match positions

In [ ]:
times = np.array(position_cols[0])
badpoints = 0

for elem in data:
    time = elem[0]
    obs_iotid = elem[2]
    foi_iotid = elem[3]
    
    temp = np.abs(times - time)
    minimum = np.min(temp)
    index = np.argmin(temp)
    
    # maximum time difference: 10 seconds
    if minimum > 10.0:
        badpoints += 1
        continue
    
    st.UpdateFoICoordinates(st_url + "/v1.0/FeaturesOfInterest('" + foi_iotid + "')", position[index][2:])

In [ ]:
badpoints